<a href="https://colab.research.google.com/github/lenyabloko/SemEval2020/blob/master/SemEval2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install transformers
!pip install tensorboardx
!pip install simpletransformers

UPLOAD FILES - Place [train.csv](https://github.com/arielsho/Subtask-1/archive/master.zip) file directly under your `gdrive/My Drive/Subtask-1/`, before uploading it here (you will need to open browser and get authentication token)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

#from google.colab import files
#uploaded = files.upload()

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-1/train.csv /content

import pandas as pd
prefix = '/content/'
train_df = pd.read_csv(prefix + 'train.csv', header=None)
train_df = train_df.drop(index=0)

# display 
train_df.head()

In [0]:
dev_df = pd.DataFrame({
    'id':train_df[0],
    'labels':train_df[1],
    'alpha':['a']*train_df.shape[0],
    'text': train_df[2].replace(r'\n', ' ', regex=True)
})

train_df = pd.DataFrame({
    'id':train_df[0],
    'labels':train_df[1],
    'alpha':['a']*train_df.shape[0],
    'text': train_df[2].replace(r'\n', ' ', regex=True)
})

dev_df.head()

In [0]:
train_df.to_csv(prefix+'train.tsv', sep='\t', index=False, header=False)
dev_df.to_csv(prefix + 'dev.tsv', sep='\t', index=False, header=False)

In [0]:
from simpletransformers.classification import ClassificationModel

from sklearn.model_selection import train_test_split

from tqdm import tqdm_notebook

args = {
   'model_type':  'roberta',
   'model_name': 'roberta-base',
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
'fp16': True,
   'fp16_opt_level': 'O1',
   'max_seq_length': 128,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 1,
   'weight_decay': 0,
   'learning_rate': 4e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
'overwrite_output_dir': False,
   'reprocess_input_data': True
}

# Create a TransformerModel
model = ClassificationModel('roberta', 'roberta-base', args=args)
#model = simpletransformers.TransformerModel('bert', 'bert-base-cased', args={'num_train_epochs': 1, 'learning_rate': 3e-5})

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

!cp /content/args.json /content/outputs
!mv /content/outputs /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-1/              